## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd

## Data Read-in

In [3]:
df = pd.read_csv('redfin_2023-11-03-12-00-40.csv')
BC_gf = gpd.read_file('countyboundary')

## Data Clean

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
df = df.dropna(subset=['SOLD DATE'])

In [6]:
# Define list of desired months (excluding current month)
desired_months = ['October']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [7]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    85
Name: count, dtype: int64
-------
$/SQUARE FEET
False    83
True      2
Name: count, dtype: int64
-------
YEAR BUILT
False    85
Name: count, dtype: int64
-------


In [51]:
# sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
# second_newest_building = sorted_df.iloc[2]
# print(second_newest_building['URL'])

In [8]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [9]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [10]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
43,PAST SALE,October-10-2023,Condo/Co-op,3160 Holiday Springs Blvd #104,Margate,FL,33063,67000.0,1,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/3160-Holiday...,Beaches MLS,F10398276,N,Y,26.265849,-80.233308
59,PAST SALE,October-12-2023,Condo/Co-op,4851 NW 21st St #211,Lauderhill,FL,33313,97000.0,1,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/4851-NW-2...,Beaches MLS,F10397915,N,Y,26.153009,-80.215618
28,PAST SALE,October-3-2023,Condo/Co-op,3690 Inverrary Dr Unit 2M,Lauderhill,FL,33319,105000.0,0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/3690-Inve...,Beaches MLS,F10395442,N,Y,26.171373,-80.230345
23,PAST SALE,October-6-2023,Condo/Co-op,2504 Antigua Ter Unit G3,Coconut Creek,FL,33066,107500.0,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coconut-Creek/2504-A...,Beaches MLS,F10397634,N,Y,26.249554,-80.184413
3,PAST SALE,October-30-2023,Condo/Co-op,6075 N Sabal Palm Blvd #313,Tamarac,FL,33319,115000.0,2,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tamarac/6075-N-Sabal...,Beaches MLS,RX-10932153,N,Y,26.199269,-80.217640
25,PAST SALE,October-6-2023,Condo/Co-op,2601 NW 48th Ter #151,Lauderdale Lakes,FL,33313,125000.0,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-Lakes/260...,MARMLS,A11454381,N,Y,26.159179,-80.215330
5,PAST SALE,October-27-2023,Condo/Co-op,1330 NW 43rd Ave #102,Lauderhill,FL,33313,127000.0,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/1330-NW-4...,Beaches MLS,F10397859,N,Y,26.143807,-80.206741
51,PAST SALE,October-19-2023,Condo/Co-op,3012 Swansea A #3012,Deerfield Beach,FL,33442,129900.0,1,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/3012...,Beaches MLS,F10397200,N,Y,26.314515,-80.129391
57,PAST SALE,October-11-2023,Condo/Co-op,2151 NE 68th St #226,Fort Lauderdale,FL,33308,130000.0,1,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2151...,Beaches MLS,F10401045,N,Y,26.210602,-80.114045
9,PAST SALE,October-27-2023,Condo/Co-op,3000 NE 16th Ave #313,Oakland Park,FL,33334,140000.0,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Oakland-Park/3000-NE...,Beaches MLS,F10400628,N,Y,26.165076,-80.125217


In [15]:
print(df_filtered['URL'].iloc[43])

https://www.redfin.com/FL/Margate/3160-Holiday-Springs-Blvd-33063/unit-104/home/187327340


In [56]:
# # Correct the prices, if needed
# df_filtered.at[23,'PRICE']=(115000)
# df_filtered.at[559,'PRICE']=(180000)

In [13]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [58]:
# # Corrections, if needed
# df_filtered.at[23,'$/SQUARE FEET']=(1150000/1019)
# df_filtered.at[559,'$/SQUARE FEET']=(180000/961)

In [14]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
43,67000.0,3160 Holiday Springs Blvd #104,Margate,92.0
23,107500.0,2504 Antigua Ter Unit G3,Coconut Creek,93.0
59,97000.0,4851 NW 21st St #211,Lauderhill,105.0
74,150000.0,1800 NW 55th Ave Unit 4U,Lauderhill,111.0
3,115000.0,6075 N Sabal Palm Blvd #313,Tamarac,119.0
29,160000.0,1817 NW 54th Ter Unit BB3,Lauderhill,119.0
47,170000.0,2981 N Nob Hill Rd #102,Sunrise,125.0
25,125000.0,2601 NW 48th Ter #151,Lauderdale Lakes,130.0
65,185000.0,4955 E Sabal Palm Blvd #306,Tamarac,136.0
5,127000.0,1330 NW 43rd Ave #102,Lauderhill,137.0


In [60]:
print(df_filtered.URL.iloc[8])

https://www.redfin.com/FL/Pembroke-Pines/711-SW-111th-Way-33025/unit-304/home/42065267


In [61]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [16]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [17]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [18]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [19]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [20]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [21]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"October 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(BC_gf,tooltip='Broward County',name='Broward County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [22]:
m.save('index.html')

## Summary Info

In [23]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [24]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [25]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [26]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/3200-NE-36th-St-33308/unit-822/home/41715110


In [27]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Las Olas Beach Club, 101 S Ft Lauderdale Bch Blvd #2701 Fort Lauderdale | Price $3,950,000 | $973 psf | Year built: 2007
Least Expensive
Holiday Springs, 3160 Holiday Springs Blvd #104 Margate | Price $67,000 | $92 psf | Year built: 1974
Highest Price Per Square Foot
Las Olas Beach Club, 101 S Ft Lauderdale Bch Blvd #2701 Fort Lauderdale | Price $3,950,000 | $973 psf | Year built: 2007
Lowest Price Per Square Foot
Holiday Springs, 3160 Holiday Springs Blvd #104 Margate | Price $67,000 | $92 psf | Year built: 1974
Newest
METROPICA NORTH TOWER ONE, 2000 Metropica Way #2403 Sunrise | Price $768,900 | $652 psf | Year built: 2020
Oldest
Coral Ridge Towers North, 3200 NE 36th St #822 Fort Lauderdale | Price $285,000 | $254 psf | Year built: 1963


## Time on Market Calculator

In [28]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/101-S-Fort-Lauderdale-Beach-Blvd-33316/unit-2701/home/42067740


In [82]:
from datetime import datetime, timedelta

date1 = datetime(2023, 6, 26) ## List (Earlier) date
date2 = datetime(2023, 9, 26) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

92


## Map URL Snagger

In [76]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [77]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/BC_condo_sales_month_ending_sep_2023


## Get Summary Data

In [78]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 820
--------
Total sale price: $251,146,176
--------
Median sale price: $235,000
--------
Max sale price: $3,725,000
--------
Min sale price: $63,000
------------------------------------------------
PSF INFO
Max price per square foot: $1,327
--------
Min price per square foot: $68
--------
Median price per square foot: $233
------------------------------------------------
CONDO AGES
Newest building: 2020.0
----------
Oldest building: 1958.0
----------
Average building age: 1980.9475609756098
